In [29]:
import argparse
import json
import logging
import os
import sys
import pickle

import dgl
import torch
import numpy as np
from ogb.nodeproppred import DglNodePropPredDataset, Evaluator
import time 
import numpy as np
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, ReduceLROnPlateau


import random
import wandb
wandb.login()

import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import SAGEConv
import tqdm
import sklearn.metrics

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

import warnings
warnings.filterwarnings("ignore")

class Model(nn.Module):
    def __init__(
        self, in_feats, n_hidden, n_classes, n_layers, dropout, activation, aggregator_type='mean'
    ):
        super(Model, self).__init__()
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_classes = n_classes
        self.layers = nn.ModuleList()
        self.layers.append(SAGEConv(in_feats, n_hidden, aggregator_type=aggregator_type))
        for _ in range(n_layers - 2):
            self.layers.append(SAGEConv(n_hidden, n_hidden, aggregator_type=aggregator_type))
        self.layers.append(SAGEConv(n_hidden, n_classes, aggregator_type=aggregator_type))
        self.dropout = nn.Dropout(dropout)
        self.activation = activation

    def forward(self, g, x):
        h = x
        for l, conv in enumerate(self.layers):
            h = conv(g, h)
            # print("self.activation = {}".format(type(self.activation)))
            if l != len(self.layers) - 1:
                h = self.activation(h)
                h = self.dropout(h)
        return h


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [30]:

def _get_data_loader(sampler, device, graph, nids, batch_size=1024):
    logger.info("Get train-val-test data loader")
    train_nids, valid_nids, test_nids = nids
    logger.info("Get train data loader")
    train_dataloader = dgl.dataloading.DataLoader(
    # The following arguments are specific to DGL's DataLoader.
    graph.subgraph(train_nids),              # The graph
    train_nids,         # The node IDs to iterate over in minibatches
    sampler,            # The neighbor sampler
    device=device,      # Put the sampled MFGs on CPU or GPU
    # The following arguments are inherited from PyTorch DataLoader.
    batch_size=batch_size,    # Batch size
    shuffle=True,       # Whether to shuffle the nodes for every epoch
    drop_last=False,    # Whether to drop the last incomplete batch
    num_workers=0       # Number of sampler processes
    )
    logger.info("Get val data loader")
    valid_dataloader = dgl.dataloading.DataLoader(
    graph.subgraph(valid_nids), valid_nids, sampler,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=0,
    device=device
    )

    logger.info("Get test data loader")
    test_dataloader = dgl.dataloading.DataLoader(
    graph.subgraph(test_nids), test_nids, sampler,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=0,
    device=device
    )

    logger.info("Train-val-test data loader created")
    
    return (train_dataloader, valid_dataloader, test_dataloader)

@torch.no_grad()
def evaluate(evaluator, predictions, labels):
    acc = evaluator.eval({
        'y_true': torch.reshape(labels, (-1, 1)),
        'y_pred': torch.reshape(predictions, (-1, 1)),
    })['acc']
    # eacc = sklearn.metrics.accuracy_score(labels, predictions)
    return acc

@torch.no_grad()
def evaluate2(logits, labels, mask):
    logits = logits[mask]
    labels = labels[mask]
    _, indices = torch.max(logits, dim=1)
    correct = torch.sum(indices == labels)
    return correct.item() * 1.0 / len(labels)

def evaluate3(indices, labels):
    # _, indices = torch.max(logits, dim=1)
    correct = torch.sum(indices == labels)
    return correct.item() * 1.0 / len(labels)



In [9]:
# def train():
config={
        "num_epochs": 10,
        "lr": 2*1e-3,
        "dropout": random.uniform(0.3, 0.6),
        "n_hidden": 1400,
        "n_layers": 3,
        "agg": "mean",
        "batch_size": 2**12,
        "budget": 1000,
        }

n_layers = config['n_layers']
n_hidden = config['n_hidden']
num_epochs = config['num_epochs']
dropout = config['dropout']
batch_size = config['batch_size']
lr = config['lr']
agg = config['agg']
budget = config['budget']

root="../dataset/"
dataset = DglNodePropPredDataset('ogbn-arxiv', root=root)
device = "cuda" if torch.cuda.is_available() else "cpu"

idx_split = dataset.get_idx_split()
train_nids = idx_split['train']
valid_nids = idx_split['valid']
test_nids = idx_split['test']

graph, node_labels = dataset[0]
graph = dgl.add_reverse_edges(graph)
graph.ndata['label'] = node_labels[:, 0]

node_features = graph.ndata['feat']
in_feats = node_features.shape[1]
n_classes = (node_labels.max() + 1).item()


In [28]:
t = graph.subgraph(train_nids).ndata['_ID']
v = graph.subgraph(valid_nids).ndata['_ID']
te = graph.subgraph(test_nids).ndata['_ID']
print(len(torch.cat((t,v,te))))
print(len(t) + len(v) + len(te))

169343
169343


In [ ]:
# sampler = dgl.dataloading.NeighborSampler([fanout for _ in range(n_layers)])
sampler = dgl.dataloading.SAINTSampler(mode='node', budget=budget)

data = _get_data_loader(sampler, device, graph, (train_nids, valid_nids, test_nids), batch_size)

train_dataloader, valid_dataloader, test_dataloader = data

activation = F.relu

model = Model(in_feats, n_hidden, n_classes, n_layers, dropout, activation, aggregator_type=agg).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler1 = CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=1, eta_min=1e-3)
scheduler2 = ReduceLROnPlateau(optimizer, mode='max', factor=0.99, patience=20, min_lr=1e-5)

evaluator = Evaluator(name='ogbn-arxiv')
best_train_acc = 0
best_val_acc = 0
best_test_acc = 0

best_model_path = 'model.pt'
best_model = None
total_time = 0

time_load = 0
time_forward = 0
time_backward = 0
total_time = 0
for epoch in range(num_epochs):
    # print("epoch = {}".format(epoch))
    model.train()
    tic = time.time()



    for step, subg in enumerate(train_dataloader):
        # print(step)
        tic_start = time.time()
        inputs = subg.ndata['feat']
        labels = subg.ndata['label']
        tic_step = time.time()
        # print("tic_step= {}".format(tic_step))
        predictions = model(subg, inputs)
        loss = F.cross_entropy(predictions, labels)
        optimizer.zero_grad()
        tic_forward = time.time()
        # print("tic_forward = {}".format(tic_forward))
        loss.backward()
        optimizer.step()
        tic_backward = time.time()
        # print("tic_backward = {}".format(tic_backward))

        time_load += tic_step - tic_start
        time_forward += tic_forward - tic_step
        time_backward += tic_backward - tic_forward

    scheduler2.step(best_val_acc)
#     scheduler1.step()
    toc = time.time()
    total_time += toc - tic

    if epoch % 5 == 0:
        model.eval()
        # print("evalua")
        train_predictions = []
        train_labels = []
        val_predictions = []
        val_labels = []
        test_predictions = []
        test_labels = []
        with torch.no_grad():


            pred = model(graph.to(device), graph.ndata['feat'].to(device))

            for subg in train_dataloader:
                inputs = subg.ndata['feat']
                train_labels.append(subg.ndata['label'])
                train_predictions.append(model(subg, inputs).argmax(1))
            train_predictions = torch.cat(train_predictions)
            train_labels = torch.cat(train_labels)
            train_acc_subgraph_sample = sklearn.metrics.accuracy_score(train_labels.cpu().numpy(), train_predictions.cpu().numpy())

            train_acc_fullgraph_no_sample = evaluate(evaluator, pred[train_nids].argmax(1), graph.ndata['label'][train_nids])

            pred_train = model(graph.subgraph(train_nids).to(device), graph.ndata['feat'][train_nids].to(device))
            train_acc_subgraph_no_sample = evaluate(evaluator, pred_train.argmax(1), graph.ndata['label'][train_nids])
            # train_acc_thru_evaltr = evaluate2(pred, graph.ndata['label'].to(device), train_nids)
            # train_acc_thru_evaltr = evaluate3(train_predictions, train_labels)

            for subg in valid_dataloader:
                inputs = subg.ndata['feat']
                val_labels.append(subg.ndata['label'])
                val_predictions.append(model(subg, inputs).argmax(1))
            val_predictions = torch.cat(val_predictions)
            val_labels = torch.cat(val_labels)
            val_acc_subgraph_sample = sklearn.metrics.accuracy_score(val_labels.cpu().numpy(), val_predictions.cpu().numpy())

            val_acc_fullgraph_no_sample = evaluate(evaluator, pred[valid_nids].argmax(1), graph.ndata['label'][valid_nids])

            pred_valid = model(graph.subgraph(valid_nids).to(device), graph.ndata['feat'][valid_nids].to(device))
            val_acc_subgraph_no_sample = evaluate(evaluator, pred_valid.argmax(1), graph.ndata['label'][valid_nids])
            # val_acc_thru_evaltr = evaluate2(pred, graph.ndata['label'].to(device), valid_nids)
            # val_acc_thru_evaltr = evaluate3(val_predictions, val_labels)


            for subg in test_dataloader:
                inputs = subg.ndata['feat']
                test_labels.append(subg.ndata['label'])
                test_predictions.append(model(subg, inputs).argmax(1))
            test_predictions = torch.cat(test_predictions)
            test_labels = torch.cat(test_labels)
            test_acc_subgraph_sample = sklearn.metrics.accuracy_score(test_labels.cpu().numpy(), test_predictions.cpu().numpy())

            test_acc_fullgraph_no_sample = evaluate(evaluator, pred[test_nids].argmax(1), graph.ndata['label'][test_nids])

            pred_test = model(graph.subgraph(test_nids).to(device), graph.ndata['feat'][test_nids].to(device))
            test_acc_subgraph_no_sample = evaluate(evaluator, pred_test.argmax(1), graph.ndata['label'][test_nids])
            # test_acc_thru_evaltr = evaluate2(pred, graph.ndata['label'].to(device), test_nids)
            # test_acc_thru_evaltr = evaluate3(test_predictions, test_labels)



            if best_val_acc < val_acc_subgraph_sample:
                best_val_acc = val_acc_subgraph_sample
                best_model = model
                best_test_acc = test_acc_subgraph_sample
                best_train_acc = train_acc_subgraph_sample
            logger.debug('Epoch {}, Train Acc {:.4f} (Best {:.4f}), Val Acc {:.4f} (Best {:.4f}), Test Acc {:.4f} (Best {:.4f})'.format(epoch, train_acc_subgraph_sample, best_train_acc, val_acc_subgraph_sample, best_val_acc, test_acc_subgraph_sample, best_test_acc))

In [7]:
val_acc, model = train()


best_test_acc,▁█
best_train_acc,▁█
best_val_acc,▁█
lr,█▁
test_acc,▁█
test_acc_fullgraph_no_sample,▁█
test_acc_subgraph_no_sample,▁█
test_diff,▁█
train_acc,▁█
train_acc_fullgraph_no_sample,▁█
train_acc_subgraph_no_sample,▁█


Get train-val-test data loader
Get train-val-test data loader
Get train data loader
Get train data loader
Get val data loader
Get val data loader
Get test data loader
Get test data loader
Train-val-test data loader created
Train-val-test data loader created
Epoch 0, Train Acc 0.4755 (Best 0.4755), Val Acc 0.3977 (Best 0.3977), Test Acc 0.3762 (Best 0.3762)
Epoch 0, Train Acc 0.4755 (Best 0.4755), Val Acc 0.3977 (Best 0.3977), Test Acc 0.3762 (Best 0.3762)
Epoch 5, Train Acc 0.6527 (Best 0.6527), Val Acc 0.5603 (Best 0.5603), Test Acc 0.5390 (Best 0.5390)
Epoch 5, Train Acc 0.6527 (Best 0.6527), Val Acc 0.5603 (Best 0.5603), Test Acc 0.5390 (Best 0.5390)
total time for 10 epochs = 1.4333500862121582
total time for 10 epochs = 1.4333500862121582
avg time per epoch = 0.14333500862121581
avg time per epoch = 0.14333500862121581
